In [1]:
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import numpy as np 
from tqdm import tqdm
import math
import os
import json 

from sklearn.decomposition import PCA
from scipy.linalg import eigh
import requests
from fredapi import Fred
from bs4 import BeautifulSoup


We will take $EUR/USD$ as an example. Suppose that we have no arbitrage opportunity, we must then have: 

$$(1 + r_{EUR})F_{EURUSD} = S_{EURUSD}(1 + r_{USD})$$

- LHS: Take 1EUR and deposit it in an EUR account, then lock in the exchange rate with a forward contract, converting back to USD. 

- RHS: Convert 1EUR to USD at spot rate and deposit it into USD account for same period of time as Forward maturity on LHS.

### Using Forward for Funding in Foreign Currency

$$(1+r_{𝑈𝑆𝐷}+r_{𝑏𝑎𝑠𝑖𝑠})=(1+𝑟_{𝐸𝑈𝑅})\frac{𝐹_{𝐸𝑈𝑅/𝑈𝑆𝐷}}{𝑆_{𝐸𝑈𝑅/𝑈𝑆𝐷}}$$



Let us now imagine someone has access to EUR currency and wants to use it to __obtain USD "funding" for a specific period of time (say 1 year)__: they know the interest rate $𝑟_{𝐸𝑈𝑅}$ at which they can raise the $EUR$ (which is their domestic currency). They also know the FX Spot between EUR and USD (i.e. $𝑆_{𝐸𝑈𝑅𝑈𝑆𝐷}$) and the 1-year FX Forward rate between $EUR$ and $USD$ (i.e. $𝐹_{𝐸𝑈𝑅𝑈𝑆𝐷}$).

In other words, they can obtain $EUR$, exchange the $EUR$ for $USD$ at spot and __using the 1-y Forward, they can lock into an implied "USD interest rate"__: the interest rate they will effectively have to "pay" to get access to USD funding for the duration of 1-year. 

Currently, the EURUSD-OIS basis is negative, indicating an extremely attractive cost of USD funding from Europe, which is indicative of excess demand for EURO funding from the US.

Indeed, In our actual case (EUR/USD), Corporate credit spreads in the euro bond market have fallen relative to those in the US dollar bond market, largely driven by ECB bond purchase programmes. In response, US firms have found it more cost-effective to issue in euros, through so-called reverse yankee bonds, and then swap the proceeds into US dollars.

### Summary: $r_{basis}$ meaning

$r_{basis}$ reflects the prevailing __demand for funding in one currency via another currency, for a fixed period of time (term). Each term (i.e. 6m, 1y, 5y, etc) would have it's own  $r_{basis}$__.

When OIS rates are used in the equations above, you can back out the Xccy-OIS basis. If Libor rates are used instead, you can back out Libor-Xccy basis.

So in conclusion, taking 6-month tenor as an example: you know the 6m EUR-OIS rate, the 6m USD-OIS rate, the EUR/USD Spot and the 6m EUR/USD Forward: when you plug all of these into the equations above, you can back out the 6m FX-OIS basis for EUR/USD (and you can do this for any other tenor or currency).

### Computation

Thus, for every tenor we have to compute:

$$r_{basis} = (1 + r_{EUR})\frac{F_{EUR/USD}}{S_{EUR/USD}} - (1+r_{USD})$$

In [10]:
# Definir el rango de fechas para recuperar el data
start_date = dt.datetime(2005, 1, 1)
end_date = dt.datetime.now()

# Recuperar el data desde FRED de la reserva federal 
df_spot = pdr.data.DataReader('DEXUSEU', 'fred', start_date, end_date)
df_spot.rename(columns = {'DEXUSEU':'EURUSD'}, inplace = True)


In [84]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Set the URL of the website to scrape
url = 'https://www.fxempire.com/currencies/eur-usd/forward-rates'

headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
# Set the tenors in days
columns = ["ON", "1D", "Spot",
           "1W", "2W", "3W",
           "1M", "2M","3M","4M","5M","6M","7M","8M","9M","10M","11M",
           "1Y", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "10Y"]

# Make a request to the website
response = requests.get(url, headers = headers)
# Parse the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')
df = pd.read_html(response.content)[0]
df['Expiration'
df[['Expiration', 'Mid' ]]


,Expiration,Mid
0,Overnight,1.09824
1,Tomorrow Next,1.09824
2,Spot Next,1.09824
3,One Week,1.09862
4,Two Weeks,1.09905
5,Three Weeks,1.09952
6,One Month,1.10006
7,Two Months,1.10197
8,Three Months,1.10361
9,Four Months,1.10526


In [60]:
pd.read_html('https://www.fxempire.com/currencies/eur-usd/forward-rates')

HTTPError: HTTP Error 403: Forbidden